# ENGRI 1120: Series Cell-free Production and Recovery of the mRNA BNT-162b2 Vaccine

#### Your team name, your names and netids go here

<img src="figs/Fig-Series-System.png" style="width:50%">

## Introduction
The project introduction goes here. Typically 2-3 paragraphs.

__Suggestions__:
* Paragraph 1: Describe the motivation and background information on the product and the vaccine 
* Paragraph 2/3: Summary of your project. Describe the number of units (don't forget the heat exchangers), details about the units’ operation, and your project’s financials. 

You can replace the figure above with your system.

## Materials and Methods

### Project Setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `C:\Users\James Chen\Documents\ENGRI 1120\ENGRI-1120-IntroToChemE-Example-Notebooks\project`
  No Changes to `C:\Users\James Chen\Documents\ENGRI 1120\ENGRI-1120-IntroToChemE-Example-Notebooks\project\Project.toml`
  No Changes to `C:\Users\James Chen\Documents\ENGRI 1120\ENGRI-1120-IntroToChemE-Example-Notebooks\project\Manifest.toml`


In [2]:
# load reqd packages and set paths -
using JLD2
using FileIO
using PrettyTables
using DataFrames
using GLPK

# setup paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT, "data");

#### Load the project code library
The call to the `include` function loads the `ENGRI-1120-Project-CodeLib.jl` library into the notebook; the library contains functions we can use during the project. In particular, it includes the function:

* The `compute_optimal_extent(stoichiometric_matrix::Array{Float64,2}, default_bounds_array::Array{Float64,2},
    species_bounds_array::Array{Float64,2}, objective_coefficient_array::Array{Float64,1}; min_flag::Bool = true) -> Tuple` function calls the [GLPK](https://www.gnu.org/software/glpk/) linear program solver. The `results` tuple contains several things, but the important ones are `calculated_flux_array`, `objective_value`, and the status/exit flags `status_flag` and `exit_flag` (which let us know if the solver successfully found a solution).
* The `build(model::Type{MSULatticeModel}; ṅₒ::Float64, L::Int64, u::Float64, d::Float64) -> MSULatticeModel` function builds a [Binary tree](https://en.wikipedia.org/wiki/Binary_tree) of Magical Separation Units (MSUs). Arguments: $ṅₒ$ denotes the species mole flow rate into the separation system, $L$ denotes the number of layers of the tree, $u$ denotes the `up` factor (split for the `up` path), and $d$ denotes the `down` factor (the split for the `down` path). This function returns the `MSULatticeModel` model, which contains the column array `data` holding the species mole flow rate for each node in the tree. 
* The `build_nodes_dictionary(levels::Int64) -> Dict{Int64,Array{Int64,1}}` function constructs a dictionary of node indexes for each level of the tree; keys are the tree levels.

In [3]:
include("ENGRI-1120-Project-CodeLib.jl");

In [4]:
# load the model file -
model = load(joinpath(_PATH_TO_DATA, "ENGRI-1120-BNT162b2-Model.jld2"))["model"]

┌ Warning: the following fields are present in type DataFrame saved in the file but not present in the type the workspace:
│ 
│ metadata
│ colmetadata
│ allnotemetadata,
│ 
│ Data in these fields will not be accessible
└ @ JLD2 C:\Users\James Chen\.julia\packages\JLD2\k9Gt0\src\data\reconstructing_datatypes.jl:196


Dict{String, Any} with 7 entries:
  "stochiometric_matrix" => [-1.0 0.0 … 0.0 -1.0; -1.0 0.0 … -1.0 0.0; … ; 1.0 …
  "list_of_reactions"    => ["TX_BNT_162b2_binding", "TX_BNT_162b2_open", "BNT_…
  "reaction_table"       => 6×7 DataFrame…
  "flux_bounds_array"    => [-1000.0 1000.0; 0.0 1000.0; … ; 0.0 1000.0; 0.0 10…
  "mRNA_sequence"        => ['C', 'U', 'C', 'U', 'U', 'A', 'U', 'U', 'U', 'G'  …
  "list_of_species"      => ["G_BNT_162b2", "T7RNAP", "M_atp_c", "M_utp_c", "M_…
  "gene_sequence"        => ['G', 'A', 'G', 'A', 'A', 'T', 'A', 'A', 'A', 'C'  …

In [5]:
# get stuff from the model data structure -
S = model["stochiometric_matrix"]; # fix the spelling in the model file
flux_bounds_array = model["flux_bounds_array"];
list_of_species = model["list_of_species"];
list_of_reactions = model["list_of_reactions"];
reaction_table = model["reaction_table"];
gene_sequence = model["gene_sequence"];

In [6]:
reaction_table

,id,forward
,String,String
1,TX_BNT_162b2_binding,G_BNT_162b2+T7RNAP
2,TX_BNT_162b2_open,G_BNT_162b2_T7RNAP_closed
3,BNT_162b2_transcription,G_BNT_162b2_T7RNAP_open+798*M_atp_c+1004*M_utp_c+1060*M_ctp_c+1312*M_gtp_c
4,mRNA_BNT_162b2_degradation,mRNA_BNT_162b2
5,RNAP_deactivation,T7RNAP
6,GENE_deactivation,G_BNT_162b2


In [7]:
# How many species and reactions?
(ℳ, ℛ) = size(S);

In [8]:
# initialize -
species_index_table_data = Array{Any,2}(undef, ℳ, 2);

# build table -
for i ∈ 1:ℳ
    species_index_table_data[i,1] = i;
    species_index_table_data[i,2] = list_of_species[i];
end

# setup header -
species_index_header_table = (["Index", "Species"]);

# build table -
pretty_table(species_index_table_data; header=species_index_header_table);

┌───────┬───────────────────────────┐
│ Index │                   Species │
├───────┼───────────────────────────┤
│     1 │               G_BNT_162b2 │
│     2 │                    T7RNAP │
│     3 │                   M_atp_c │
│     4 │                   M_utp_c │
│     5 │                   M_ctp_c │
│     6 │                   M_gtp_c │
│     7 │            mRNA_BNT_162b2 │
│     8 │                   M_ppi_c │
│     9 │                   M_amp_c │
│    10 │                   M_ump_c │
│    11 │                   M_cmp_c │
│    12 │                   M_gmp_c │
│    13 │           T7RNAP_inactive │
│    14 │      G_BNT_162b2_inactive │
│    15 │ G_BNT_162b2_T7RNAP_closed │
│    16 │   G_BNT_162b2_T7RNAP_open │
└───────┴───────────────────────────┘


#### Setup the constants, feed rates and compositions

In [97]:
# how many chips in series?
number_of_chips = 10;

# what fraction of the mRNA degrades?
δ = 0.10;

# MSU split ratio -
θ = 0.90;

# Setup constants for transcription -
L = length(gene_sequence);
K = 0.116; # saturation constant units: μmol/L; Source: ACS Synth. Biol. 2018, 7, 8, 1844–1857 https://doi.org/10.1021/acssynbio.7b00465
v̇ₜ = (90.0)*(60); # units: nt/s; Source: BIND: 111871
u = 0.95; # u-factor; Source: ACS Synth. Biol. 2018, 7, 8, 1844–1857 https://doi.org/10.1021/acssynbio.7b00465

# volume -
V = 100.0*(1/1e6); # liquid reaction volume on each chip units: L

# Stock solution: PURExpress -> fed into chips by pump 2 (stream 2)
# PURExpress flows into the chip in stream 2
T7RNAP = 100.0;          # concentration in PURExpress units: μmol/L
M_atp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_utp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_ctp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_gtp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L

# Stock solution: DNA -> feed into splitter by pump 1 and then into chip (always stream 1)
G_BNT_162b2 = 100.0;     # gene concentration in stock solution units: μmol/L

# Volumetric flow rates from the pump *into* the splitter unit or chip - our base case will 1 ml/min; thus, let's
# scale by the number of chips
Ḟ₁ = number_of_chips*1000.0*(1/1e6); # volumetric flow rate of syringe pump 1 units: L/min
Ḟ₂ = 200*number_of_chips*1000.0*(1/1e6); # volumetric flow rate of syringe pump 2 units: L/min

# stuff for the tables -
current_table_counter = 0; # do not change me

#### Specify the inputs streams

##### a) Specify the composition of feed stream 1
By default, this stream contains only the gene encoding the mRNA BNT-162b2 product. The mainstream is split into sub-streams that are fed into each chip.

In [33]:
# setup feed compostions for feed stream 1 
ṅ₁ = zeros(ℳ); # default is zero, correct specific values -
ṅ₁[1] = G_BNT_162b2*Ḟ₁*(1/number_of_chips); # units: μmol/min

##### b) Specify the composition of feed stream 2
By default, feed stream 2 contains the [PURExpress](https://www.neb.com/products/e6800-purexpress-invitro-protein-synthesis-kit#Product%20Information), which has everything we need to make our mRNA product of interest _expect_ the linear DNA. 

In [34]:
# setup feed compostions for feed stream 2: this feed goes into chip 1
ṅ₂ = zeros(ℳ); # default is zero, then correct specific values -
ṅ₂[2] = T7RNAP*Ḟ₂;  # units: μmol/min
ṅ₂[3] = M_atp_c*Ḟ₂; # units: μmol/min
ṅ₂[4] = M_utp_c*Ḟ₂; # units: μmol/min
ṅ₂[5] = M_ctp_c*Ḟ₂; # units: μmol/min
ṅ₂[6] = M_gtp_c*Ḟ₂; # units: μmol/min

### Flux balance analysis setup
* Describe the flux balance analysis method. 
* How did your group update the default species and flux bounds to model ths process?
* What objective function did you choose?

#### Species bounds

In [35]:
# species bounds array -
species_bounds_array = [-(ṅ₁ .+ ṅ₂) 10000.0*ones(ℳ,1)];

#### Reaction bounds
In the series case, reaction bounds will change for every chip. Let's set a default upper bound and then update the other bounds below. If you are a brave soul, you can alter the per-chip bounds, but you don't need to do so.

In [36]:
# initialize -
flux_bounds_array = zeros(ℛ,2);
flux_bounds_array[:,2] .= 1000.0; # large default upper bound

In [37]:
# initialize -
reaction_index_table_data = Array{Any,2}(undef, ℛ, 4);

# build table -
for i ∈ 1:ℛ
    reaction_index_table_data[i,1] = i;
    reaction_index_table_data[i,2] = list_of_reactions[i];
    reaction_index_table_data[i,3] = flux_bounds_array[i,1];
    reaction_index_table_data[i,4] = flux_bounds_array[i,2];
end

# setup title string -
reaction_table_title = "Table $(current_table_counter+=1): Reaction index-name mapping table."

# setup header -
reaction_index_header_table = (["Index", "Reaction", "lower bound", "upper bound"], ["", "", "μmol/min", "μmol/min"]);

# build table -
pretty_table(reaction_index_table_data, title=reaction_table_title; header=reaction_index_header_table);

Table 2: Reaction index-name mapping table.
┌───────┬────────────────────────────┬─────────────┬─────────────┐
│ Index │                   Reaction │ lower bound │ upper bound │
│       │                            │    μmol/min │    μmol/min │
├───────┼────────────────────────────┼─────────────┼─────────────┤
│     1 │       TX_BNT_162b2_binding │         0.0 │      1000.0 │
│     2 │          TX_BNT_162b2_open │         0.0 │      1000.0 │
│     3 │    BNT_162b2_transcription │         0.0 │      1000.0 │
│     4 │ mRNA_BNT_162b2_degradation │         0.0 │      1000.0 │
│     5 │          RNAP_deactivation │         0.0 │      1000.0 │
│     6 │          GENE_deactivation │         0.0 │      1000.0 │
└───────┴────────────────────────────┴─────────────┴─────────────┘


#### Objective coefficient array
* What is the objective you used?

In [38]:
# setup the objective coefficient array -
obj_vector = zeros(ℛ);
obj_vector[3] = -1; # why negative?

## Results and Discussion 

### Compute the optimal extent of reaction and exit stream for first chip

In [98]:
# initialize some space to store tmp soln -
tmp_sim_storage_space = Dict{Int64,Tuple{Array{Float64,1},Array{Float64,1}}}();

# compute -
ṅ₃ = zeros(ℳ);
for chip_index ∈ 1:number_of_chips
    
    if (chip_index == 1)
        ṅ_chip = ṅ₂
    else
        ṅ_chip = ṅ₃
    end
        
    # update the bounds model -
    F̂₁ = ((chip_index/number_of_chips)*Ḟ₁)/(Ḟ₂+(chip_index/number_of_chips)*Ḟ₁)
    F̂₂ = Ḟ₂/(Ḟ₂+(chip_index/number_of_chips)*Ḟ₁);
    
    # Estimate the RNAP and GENE concentration -
    Rₜ = T7RNAP*F̂₂;                          # effective RNAP concentratation on the chip for the bounds units: μmol/L
    GENE = G_BNT_162b2*F̂₁ + G_BNT_162b2*F̂₂  # effective GENE concentratation on the chip for the bounds units: μmol/L
    flux_bounds_array[3,:] .= Rₜ*(v̇ₜ/L)*u*(GENE/(K+GENE))*V; # equality constraint

    # Setup bound for degradation (lower bound)
    flux_bounds_array[4,1] = δ*flux_bounds_array[3,1];
    
    # compute the optimal flux, and then estimate the output on the chip
    result = compute_optimal_extent(S, flux_bounds_array, species_bounds_array, obj_vector);
    ϵ̇ = result.calculated_flux_array;
    ṅ₃ = (ṅ₁ + ṅ_chip) + S*ϵ̇;   # compute the output from the chip -
    
    # now, the outlet from chip i-1 is the input to chip 1 (in stream 2), but stream 1 stays the same
    species_bounds_array = [-(ṅ₁ .+ ṅ_chip) 10000.0*ones(ℳ,1)];
    
    # grab -
    tmp_sim_storage_space[chip_index] = (ṅ₃,ϵ̇);
end

In [99]:
system_flow_table_data = Array{Any,2}(undef, ℳ, number_of_chips+2);

# populate the table -
for i ∈ 1:ℳ
    system_flow_table_data[i,1] = list_of_species[i];
    system_flow_table_data[i,2] = i;
    
    for chip_index ∈ 1:number_of_chips
        ṅ₃ = tmp_sim_storage_space[chip_index][1];
        system_flow_table_data[i,chip_index+2] = round(ṅ₃[i], digits=3);
    end
end

# build header -
# names 
flow_table_name_row = Array{String,1}()
push!(flow_table_name_row, "Species");
push!(flow_table_name_row, "index i")
for chip_index ∈ 1:number_of_chips
    push!(flow_table_name_row, "Chip $(chip_index)")
end

# units
flow_table_units_row = Array{String,1}()
push!(flow_table_units_row, "");
push!(flow_table_units_row, "")
for chip_index ∈ 1:number_of_chips
    push!(flow_table_units_row, "μmol/min")
end


# title -
flow_table_title = "Table $(current_table_counter+=1): Chip species mole flow rates table"

# header -
flux_table_header = (flow_table_name_row, flow_table_units_row)

# show -
pretty_table(system_flow_table_data, title=flow_table_title; header=flux_table_header)

Table 1: Chip species mole flow rates table
┌───────────────────────────┬─────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│                   Species │ index i │   Chip 1 │   Chip 2 │   Chip 3 │   Chip 4 │   Chip 5 │   Chip 6 │   Chip 7 │   Chip 8 │   Chip 9 │  Chip 10 │
│                           │         │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │ μmol/min │
├───────────────────────────┼─────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│               G_BNT_162b2 │       1 │    0.001 │    0.002 │    0.003 │    0.004 │    0.005 │    0.006 │    0.007 │    0.008 │    0.009 │     0.01 │
│                    T7RNAP │       2 │      2.0 │      2.0 │      2.0 │      2.0 │      2.0 │      2.0 │      2.0 │      2.0 │      2.0 │      2.0 │
│                   M_atp_c │       3 │  1990.21 │  1980

In [18]:
# this will print the last few chips (in case the table formatting gets nutty)
system_flow_table_data

16×4 Matrix{Any}:
 "G_BNT_162b2"                 1    0.001    0.002
 "T7RNAP"                      2    0.2      0.2
 "M_atp_c"                     3  194.143  189.749
 "M_utp_c"                     4  192.63   187.103
 "M_ctp_c"                     5  192.219  186.384
 "M_gtp_c"                     6  190.37   183.147
 "mRNA_BNT_162b2"              7    0.007    0.012
 "M_ppi_c"                     8   30.638   53.616
 "M_amp_c"                     9    0.586    1.025
 "M_ump_c"                    10    0.737    1.29
 "M_cmp_c"                    11    0.778    1.362
 "M_gmp_c"                    12    0.963    1.685
 "T7RNAP_inactive"            13    0.0      0.0
 "G_BNT_162b2_inactive"       14    0.0      0.0
 "G_BNT_162b2_T7RNAP_closed"  15    0.0      0.0
 "G_BNT_162b2_T7RNAP_open"    16    0.0      0.0

### Design a downstream seperation using Magical Seperator Units (MSU)

In [100]:
# default -
ṅ₃ = tmp_sim_storage_space[number_of_chips][1]

# build a downstream seperation process with this number of levels:
number_of_levels = 2; # includes zero

In [101]:
# initialize -
tmp_storage_dict = Dict{Int64, MSULatticeModel}();

# is_product_vector -
is_product_vector = zeros(ℳ);
is_product_vector[7] = 1;

# compute the composition array -
for i ∈ 1:ℳ

    if (is_product_vector[i] == 1)
        msu_lattice_model = build(MSULatticeModel; ṅₒ = ṅ₃[i], L = number_of_levels , u = θ, d = (1 - θ));
    else
        msu_lattice_model = build(MSULatticeModel; ṅₒ = ṅ₃[i], L = number_of_levels , u = (1 - θ), d = θ);
    end
    
    # grab -
    tmp_storage_dict[i] = msu_lattice_model;
end

# grab the leaves -
nodes_dict = build_nodes_dictionary(number_of_levels);
children_dict = build_children_dictionary(nodes_dict);
tree_leaves = nodes_dict[number_of_levels-1];

# build a composition array -
number_of_nodes = length(tree_leaves);
composition_array = Array{Float64,2}(undef, number_of_nodes, ℳ);
for i ∈ 1:ℳ
    data = tmp_storage_dict[i].data;
    for j ∈ 1:number_of_nodes
        composition_array[j,i] = data[tree_leaves[j]]
    end
end

# make a pretty table and show the leaves of the tree -

# initialize -
sep_tree_flow_table_data = Array{Any,2}(undef, ℳ, length(tree_leaves) + 3)
for i ∈ 1:ℳ
    sep_tree_flow_table_data[i,1] = list_of_species[i];
    sep_tree_flow_table_data[i,2] = i;
    sep_tree_flow_table_data[i,3] = ṅ₃[i] # put node 0 in table -
        
    for j ∈ 1:length(tree_leaves)
        sep_tree_flow_table_data[i,3+j] = composition_array[j,i]
    end
end

# labels row -
label_row = Array{String,1}();
push!(label_row,"Species");
push!(label_row,"index i")
push!(label_row,"N0")
for j ∈ 1:length(tree_leaves)
    push!(label_row, "N$(tree_leaves[j])");
end

# units row -
units_row = Array{String,1}();
push!(units_row, "");
push!(units_row, "");
for j ∈ 1:length(tree_leaves)+1
    push!(units_row, "μmol/min");
end

# header -
sep_tree_flow_table_header = (label_row, units_row);

# set title -
title = "Table $(current_table_counter+=1): Magical Seperator Unit (MSU) flow table; N0 denotes the feed while N⋆ denotes the leaves of the tree."

# show -
pretty_table(sep_tree_flow_table_data, title=title; header=sep_tree_flow_table_header)

Table 2: Magical Seperator Unit (MSU) flow table; N0 denotes the feed while N⋆ denotes the leaves of the tree.
┌───────────────────────────┬─────────┬──────────┬───────────┬───────────┐
│                   Species │ index i │       N0 │        N2 │        N3 │
│                           │         │ μmol/min │  μmol/min │  μmol/min │
├───────────────────────────┼─────────┼──────────┼───────────┼───────────┤
│               G_BNT_162b2 │       1 │     0.01 │     0.001 │     0.009 │
│                    T7RNAP │       2 │      2.0 │       0.2 │       1.8 │
│                   M_atp_c │       3 │  1902.33 │   190.233 │    1712.1 │
│                   M_utp_c │       4 │  1877.11 │   187.711 │    1689.4 │
│                   M_ctp_c │       5 │  1870.26 │   187.026 │   1683.23 │
│                   M_gtp_c │       6 │  1839.42 │   183.942 │   1655.48 │
│            mRNA_BNT_162b2 │       7 │ 0.110156 │ 0.0991404 │ 0.0110156 │
│                   M_ppi_c │       8 │  510.879 │   51.0879 │  

In [22]:
# build mol frac composition table -

# construct mol frac array -
mol_frac_array = Array{Float64,2}(undef, ℳ, length(tree_leaves)+1);

# node 0 -
ṅ₃_total = sum(ṅ₃);
for i ∈ 1:ℳ
    mol_frac_array[i,1] = ṅ₃[i]*(1/ṅ₃_total);    
end

# get the sums along rows -
ṅ_total = sum(composition_array,dims = 2);
for node ∈ 1:length(tree_leaves)
    for i ∈ 1:ℳ
        mol_frac_array[i,node+1] = composition_array[node,i]*(1/ṅ_total[node]);
    end
end

# initialize -
sep_tree_mol_frac_table_data = Array{Any,2}(undef, ℳ, length(tree_leaves) + 3)
for i ∈ 1:ℳ
    sep_tree_mol_frac_table_data[i,1] = list_of_species[i];
    sep_tree_mol_frac_table_data[i,2] = i;
    sep_tree_mol_frac_table_data[i,3] = round(mol_frac_array[i,1], digits=4) # put node 0 in table -
        
    for j ∈ 1:length(tree_leaves)
        sep_tree_mol_frac_table_data[i, 3+j] = round(mol_frac_array[i,j+1], digits=4)
    end
end

# labels row -
label_mft_row = Array{String,1}();
push!(label_mft_row,"Species");
push!(label_mft_row,"index i")
push!(label_mft_row,"N0")
for j ∈ 1:length(tree_leaves)
    push!(label_mft_row, "N$(tree_leaves[j])");
end

# units row -
units_mft_row = Array{String,1}();
push!(units_mft_row, "");
push!(units_mft_row, "");
for j ∈ 1:length(tree_leaves)+1
    push!(units_mft_row, "mole frac");
end

# header -
sep_tree_mft_table_header = (label_mft_row, units_mft_row);

# set title -
title_mft = "Table $(current_table_counter+=1): Magical Seperator Unit (MSU) composition table."

# show -
pretty_table(sep_tree_mol_frac_table_data, title=title_mft; header=sep_tree_mft_table_header)

Table 5: Magical Seperator Unit (MSU) composition table.
┌───────────────────────────┬─────────┬───────────┬───────────┬───────────┐
│                   Species │ index i │        N0 │        N2 │        N3 │
│                           │         │ mole frac │ mole frac │ mole frac │
├───────────────────────────┼─────────┼───────────┼───────────┼───────────┤
│               G_BNT_162b2 │       1 │       0.0 │       0.0 │       0.0 │
│                    T7RNAP │       2 │    0.0002 │    0.0002 │    0.0002 │
│                   M_atp_c │       3 │    0.2355 │    0.2355 │    0.2355 │
│                   M_utp_c │       4 │    0.2323 │    0.2322 │    0.2323 │
│                   M_ctp_c │       5 │    0.2314 │    0.2313 │    0.2314 │
│                   M_gtp_c │       6 │    0.2273 │    0.2273 │    0.2274 │
│            mRNA_BNT_162b2 │       7 │       0.0 │    0.0001 │       0.0 │
│                   M_ppi_c │       8 │    0.0666 │    0.0665 │    0.0666 │
│                   M_amp_c │  

### Financial analysis

* Compute the [Net Present Value](https://varnerlab.github.io/ENGRI-1120-IntroToChemE-Book/chapter-1-dir/money-balances.html#net-present-value-npv) of your design using the cost spreadsheet on canvas.
* What price for the mRNA product do we need to set to have a NPV equal to zero?
* How could you improve the financial performance/NPV of your design?

## Summary and conclusions
* Fill me in, see rubric.

## References and Additional Resources
* Put any references are additional citations here